In [4]:
from utils.complete import create


import inspect

def select_and_call_function(user_input: str, function_list: list):
    prompts = []
    for function in function_list:
        source = inspect.getsource(function)
        docstring = function.__doc__ if function.__doc__ else "No docstring provided."
        prompts.append(f"Function: {function.__name__}\nDocstring: {docstring.strip()}\nSource:\n{source}")

    combined_prompt = "\n\n".join(prompts) + f"\n\nUser input: {user_input}\nWhich function should be called?"
    selected_function_name = interpret_with_openai(combined_prompt)

    # Find and call the selected function
    for function in function_list:
        if function.__name__ == selected_function_name:
            return function()

    raise ValueError("No suitable function found.")


def interpret_with_openai(prompt: str) -> str:
    """
    Uses OpenAI to interpret which function to call based on the given prompt.

    Args:
    prompt (str): The prompt to send to OpenAI.

    Returns:
    str: The name of the function to call.
    """
    print(prompt)
    response = create(prompt=prompt)  # Assuming 'create' is a wrapper for OpenAI API call
    print(response)
    return response.strip()

def function_a():
    """This function does something related to task A."""
    # Function implementation
    print("Function A called")

def function_b():
    """This function handles task B."""
    # Function implementation
    print("Function B called")

functions = [function_a, function_b]
user_input = "I need to handle task B"

# Call the function selector
select_and_call_function(user_input, functions)


Function: function_a
Docstring: This function does something related to task A.
Source:
def function_a():
    """This function does something related to task A."""
    # Function implementation
    print("Function A called")


Function: function_b
Docstring: This function handles task B.
Source:
def function_b():
    """This function handles task B."""
    # Function implementation
    print("Function B called")


User input: I need to handle task B
Which function should be called?
function_b
Function B called
